In [ ]:
import nltk.corpus
nltk.download('abc')
nltk.download('punkt')

[nltk_data] Downloading package abc to /root/nltk_data...
[nltk_data]   Unzipping corpora/abc.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
sentences = nltk.corpus.abc.sents()

In [ ]:
len([word for sent in sentences for word in sent])

766811

In [ ]:
from collections import Counter
import regex
import numpy as np

class LM:
    def __init__(self, n_type):
        """
        type: ['bigram', 'trigram']
        """
        self.n_type = n_type
        if self.n_type not in ['bigram', 'trigram']:
            raise Exception("type must be either 'bigram' or 'trigram'")
        
        self.unique_voc_ = None  # Vocabulary of all the unique words of the corpus
        self.unigram_voc_ = None  # Unigram vocabulary containing words with at least 10 counts
        self.bigram_voc_ = None  # Bigram vocabulary
        self.trigram_voc_ = None  # Trigram vocabulary

        self.previous_words = None  # Number of bigrams where the second element is the keys of the dictionary
        self.following_words = None  # Number of bigrams where the first element is the keys of the dictionary
         
    
    @staticmethod
    def word_cleaner(words):  # remove any character that is not a letter or number, skip the special tokens
        """
        input must be a list of words
        """
        allowed = ['*UNK*', '*start*', '*start1*', '*start2*', '*end*']
        return list(filter(None, 
            [word if word in allowed else regex.sub(r"[^a-zA-Z0-9]", "", word).lower() if regex.sub(r"[^a-zA-Z0-9]", "", word) != "" else "" for word in words]))


    @staticmethod
    def bigram(words):  # create bigrams
        return list(zip(words,words[1:]))
    

    @staticmethod
    def trigram(words):  # create trigrams
        return list(zip(words, words[1:], words[2:]))


    @staticmethod
    def sent_preprocessing(sentences, ngram_padding, padding=True):
        """
        Cleans and pads the sentences according to the type of the model. Input must be 
        a list of lists.
        """
        clean_sents = [LM.word_cleaner(sent) for sent in sentences]  # clean words
        
        if padding:
            if ngram_padding == 2:  # padding for bigrams (start with *start* and end with *end*)
                padded_sentences = [['*start*'] + sent + ['*end*'] for sent in clean_sents]
            elif ngram_padding == 3:  # trigram padding (each sentence begins with *start1* and *start2* and ends with *end*)
                padded_sentences = [['*start1*'] + ['*start2*'] + sent + ['*end*'] for sent in clean_sents]
            else:
                raise Exception('ngram_padding must be equal to 2 or 3')
        else:
            padded_sentences = clean_sents

        return padded_sentences

    
    @staticmethod
    def unk_replacer(words, voc):
        """
        Replaces the words with a count < 10 with the special token *UNK*. Input must be a list of tokens
        """
        return [word if voc[word] > 10 and (len(word) > 1 or word in ['i', 'a']) else "*UNK*" for word in words]
    

    def train(self, train_sentences):
        """
        train_sentences must be a list of list, where each list is a different sentence.
        """
        if self.n_type == 'bigram':
            train_sents = LM.sent_preprocessing(train_sentences, 2)  # preprocess the sentences (cleans words and adds padding)
            total_words = [word for sent in train_sents for word in sent]  # flatten list
            
            self.unique_voc_ = Counter(total_words)  # Dictionary with all the counts of the unique tokens

            train_words = LM.unk_replacer(total_words, self.unique_voc_)  # Replace low-count tokens with *UNK*
            self.unigram_voc_ = Counter(train_words)  # Create the unigram vocabulary

            self.bigram_voc_ = Counter(LM.bigram(train_words))  # create a vocabulary of all bigrams
            del self.bigram_voc_[('*end*', '*start*')]

            self.previous_words = Counter(bi[1] for bi in list(self.bigram_voc_.keys()))  # create the voc with the number of previous words
            self.following_words = Counter(bi[0] for bi in list(self.bigram_voc_.keys()))  # create the voc with the number of following words
        else:
            train_sents = LM.sent_preprocessing(train_sentences, 3)  # preprocess sentences for a trigram model
            total_words = [word for sent in train_sents for word in sent]  # flatten list

            self.unique_voc_ = Counter(total_words)

            train_words = LM.unk_replacer(total_words, self.unique_voc_)
            self.unigram_voc_ = Counter(train_words)
            
            self.bigram_voc_ = Counter(LM.bigram(train_words))  # creates bigram voc
            self.trigram_voc_ = Counter(LM.trigram(train_words))  # create trigram voc
            del self.trigram_voc_[('*end*', '*start1*', '*start2*')]
            del self.bigram_voc_[('*end*', '*start1*')]


    def estimate_ngram_prob(self, c_ngram, a=1):
        """
        Calculates the probability of a ngram using Add-a smoothing. 
        ngram must be a bigram or a trigram in a tuple.
        """        
        if a<0 and a>1:
            raise Exception('a must be in [0,1]')

        if self.n_type == 'bigram':
            if len(c_ngram) == 2:
                prob = (self.bigram_voc_[c_ngram] + a) / (self.unigram_voc_[c_ngram[0]] + a*np.abs(len(self.unigram_voc_)))  # add-a probability
            else:
                raise Exception('input is not a bigram')
        else:
            if len(c_ngram) == 3:  # same as above, but for trigrams
                prob = (self.trigram_voc_[c_ngram] + a) / (self.bigram_voc_[c_ngram[:2]] + a*np.abs(len(self.unigram_voc_)))  # |V| unigram bigram or trigram?
            else:
                raise Exception('input is not a trigram')
        return prob


    def kn_prob(self, ngram):
        """
        Calculate the probability of a bigram using Interpolated K-N smoothing.
        """
        if len(ngram) != 2:
            raise Exception('Interpolated K-N Smoothing is only available for bigrams')

        if self.bigram_voc_[ngram] == 1:  # If the count is equal to 1, then steal 0.5
            d = 0.5
        else:  # else steal 0.75
            d = 0.75

        highest_term = max((self.bigram_voc_[ngram] - d), 0) / self.unigram_voc_[ngram[0]]  # first term of the interpolated kn-smoothing formula
        
        following_words = self.following_words[ngram[0]]  # number of words that follow the first element of the bigram

        lamda = (d / self.unigram_voc_[ngram[0]]) * following_words  # interpolation weight

        previous_words = self.previous_words[ngram[1]]  # number of words preceding  the second element of the bigram
        p_cont = previous_words / len(self.bigram_voc_.keys())  # Pcontinuation

        p_kn = highest_term + lamda*p_cont
        return p_kn


    def estimate_sent_prob(self, test_sents, padding=True, a=1, smoothing='add_a', ngram_count=False):
        """
        Input must be a list of lists, where each list is a sentence. Available smoothers for bigram: 'add_a', 'kn'.
        """
        if ngram_count not in [True, False]:
            raise Exception('unk can be either se to True or False')

        if a<0 and a>1:
            raise Exception('a must be in [0,1]')

        if smoothing not in ['add_a', 'kn']:
            if self.n_type == 'trigram':
                if smoothing == 'kn':
                    raise Exception('K-N smoothing is only available for bigrams')
            raise Exception('Available smoothers: [add_a, kn]')
        
        sent_probs = []
        total_prob = 0
        ngrams_count = []

        if self.n_type == 'bigram':
            if not padding:
                test_sents = LM.sent_preprocessing(test_sents, 2, padding=False)  # do not pad sentences
            else:
                test_sents = LM.sent_preprocessing(test_sents, 2, padding=True)  # preprocess sentences in order to estimate their probabilities
             
            for sent in test_sents:
                ngrams_count.append(len(LM.bigram(sent)))  # number of bigrams in sentence(useful for cross-entropy calculation)

                for bi in LM.bigram(sent):
                    if smoothing == 'add_a':
                        total_prob += np.log2(self.estimate_ngram_prob(bi, a)) # on every iteration, it adds the log probability of each bigram 
                    else:
                        total_prob += np.log2(self.kn_prob(bi))  # use kn

                sent_probs.append(total_prob)  # if multiple sentences are given as an input, it creates a list with the log prob of each sentence
                total_prob = 0  # resets total prob to 0, in order to calculate the probability of the next sentence
        else:
            test_sents = LM.sent_preprocessing(test_sents, 3)

            for sent in test_sents:
                ngrams_count.append(len(LM.trigram(sent)))

                for tri in LM.trigram(sent):
                    total_prob += np.log2(self.estimate_ngram_prob(tri, a))  # include P(*end*|...)
                    
                sent_probs.append(total_prob)  # exp to get probabilities?
                total_prob = 0

        if ngram_count is False:
            return sent_probs
        else:
            return sent_probs, ngrams_count


    def entr_perp(self, test_sents, a=1, smoothing='add_a'):
        """
        Calculates the Cross-entropy and Perplexity of a list of sentences
        """
        test = [LM.unk_replacer(sent, self.unique_voc_) for sent in test_sents]  # replace low-count words with *UNK*
        probs, ngrams_count = self.estimate_sent_prob(test, smoothing=smoothing, a=a, ngram_count=True)  # get the probability of the sentence and its ngram counts
        
        hc = -sum(probs) / sum(ngrams_count)  # calculate cross-entropy
        perp = 2**hc  # calculate perplexity

        return hc, perp

First, we have to create an instance of the model. if n_type is equal to 'bigram', then an instance of a bigram model is created. If n_type is set to trigram, then a 'trigram' model is created.

In [ ]:
b_model = LM(n_type='bigram')
tri_model = LM(n_type='trigram')

Next, we have to train the model. The input must be a list of lists, where each list is a sentence. All the cleaning and padding is handled by the model, so we just have to feed it with raw sentences (for example, for the bigram model, if we pass to the train method the sentence ['ThI,!s', 'iS', '.A.', 'tE==++sT'], the model will transform the sentence to ['* start *', 'this', 'is', 'a', 'test', '* end *'] and it will create all the necessary vocabularies.)

Once the model is trained, then the following methods can be called:
Let b_model be an instance of a bigram model:
 - b_model.unigram_voc_ -> this will return the vocabulary of all unigrams
 - b_model.bigram_voc_ -> this will return the vocabulary of all bigrams (for the trigram model we would have to make an instance of a trigram model using LM(n_type='trigram'))
 - b_model.estimate_ngram_prob() -> this will calculate the probability of a given bigram (or trigram if we use the trigram model) using Add-a smoothing, where the hyper-parameter 'a' can be tuned in order to achieve the lowest possible Cross-Entropy
 - b_model.kn_prob() -> this will calculate the probability of a given bigram using the interpolated Kneser-Ney smoothing, where the constant D is equal to 0.5 for bigrams with a count of 1 and 0.75 for the rest.
 - b_model.estimate_sent_prob() -> this will calculate the log probabilities of all the given sentences. If more than one sentence is given as an input, then it will return a list with the probabilities of each sentence (which could then be summed and thus, calculate the total log probability of ,eg, the test corpus). There are two available smoothers for bigrams. Add-a smoothing and Interpolated Kneser-Ney smoothing.
 - b_model.entr_perp() -> this will return the Cross-Entropy and Perplexity of the test set

Some other static methods can be called (they don't require an instance creation):
 - Model.word_cleaner() -> this will clean any list of words. It will remove any character that is not a letter or number, and it will apply lower case to all the words
 - Model.sent_preprocessing() -> this will clean and pad any sentence that is given as an input
 - Model.bigram() -> this will create bigrams of a given sentence
 - Model.trigram() -> this will create trigrams of a given sentence

In [ ]:
b_model.train(sentences)  # train model

In [ ]:
len(b_model.unique_voc_)

27652

Estimating the probability of a bigram:

In [ ]:
laplace = b_model.estimate_ngram_prob(('this', 'is'))
kn = b_model.kn_prob(('this', 'is'))
print(f"Add-a smoothing (with a=1) probability: {laplace}\nK-N probability: {kn}")

Add-a smoothing (with a=1) probability: 0.04117362955807776
K-N probability: 0.12414558851175722


Estimating the log probability of a sentence using laplace:

In [ ]:
b_model.estimate_sent_prob([['this', 'is', 'a', 'test']], smoothing='add_a')  

[-34.326750073789114]

Estimating the log probability of a sentence using kn:

In [ ]:
b_model.estimate_sent_prob([['this', 'is', 'a', 'test']], smoothing='kn')  

[-26.54752395643943]

Calling the unigram vocabulary attribute:

In [ ]:
b_model.unigram_voc_

Counter({'*start*': 29059,
         'pm': 19,
         'denies': 22,
         'knowledge': 54,
         'of': 19307,
         'awb': 546,
         'kickbacks': 26,
         'the': 41634,
         'prime': 109,
         'minister': 337,
         'has': 3492,
         'denied': 13,
         'he': 3999,
         'knew': 29,
         'was': 2018,
         'paying': 37,
         'to': 18671,
         'iraq': 107,
         'despite': 191,
         'writing': 37,
         'wheat': 676,
         'exporter': 78,
         'asking': 47,
         'be': 4292,
         'kept': 47,
         'fully': 25,
         'informed': 16,
         'on': 4258,
         'sales': 142,
         '*end*': 29059,
         'letters': 41,
         'from': 4101,
         'john': 216,
         'howard': 70,
         'and': 14895,
         'deputy': 46,
         'mark': 119,
         'vaile': 68,
         'have': 4288,
         'been': 2163,
         'released': 128,
         'by': 2843,
         'cole': 144,
         'inq

Calling the bigram vocabulary output:

In [ ]:
b_model.bigram_voc_

Counter({('*start*', 'pm'): 9,
         ('pm', 'denies'): 1,
         ('denies', 'knowledge'): 1,
         ('knowledge', 'of'): 18,
         ('of', 'awb'): 36,
         ('awb', 'kickbacks'): 5,
         ('kickbacks', 'the'): 1,
         ('the', 'prime'): 40,
         ('prime', 'minister'): 91,
         ('minister', 'has'): 5,
         ('has', 'denied'): 7,
         ('denied', 'he'): 2,
         ('he', 'knew'): 4,
         ('knew', 'awb'): 2,
         ('awb', 'was'): 9,
         ('was', 'paying'): 3,
         ('paying', 'kickbacks'): 2,
         ('kickbacks', 'to'): 12,
         ('to', 'iraq'): 32,
         ('iraq', 'despite'): 1,
         ('despite', 'writing'): 1,
         ('writing', 'to'): 2,
         ('to', 'the'): 1469,
         ('the', 'wheat'): 75,
         ('wheat', 'exporter'): 49,
         ('exporter', 'asking'): 1,
         ('asking', 'to'): 1,
         ('to', 'be'): 1215,
         ('be', 'kept'): 7,
         ('kept', 'fully'): 1,
         ('fully', 'informed'): 1,
         

All of the above can be done with a trigram model. We just have to create an instance of it: tri_model = Model(type='trigram')

### Hyper-parameter Tuning

In [ ]:
from sklearn.model_selection import train_test_split

train_sents, test_set, _, _ = train_test_split(sentences, sentences, test_size=0.2, random_state=42)  # keep test set 
train_set, dev_set, _, _ = train_test_split(train_sents, train_sents, test_size=0.1, random_state=42)  # split the train set to dev and train

In [ ]:
bi_model = LM(n_type='bigram')
bi_model.train(train_set)

tri_model = LM(n_type='trigram')
tri_model.train(train_set)

In [ ]:
a = []
cr_entr = []

for i in np.arange(0.001, 1.001, 0.001):
    hc, pp = bi_model.entr_perp(dev_set, a=i)
    a.append(i)
    cr_entr.append(hc)

print(f'Best a for the bigram model: {a[np.argmin(cr_entr)]}')

Best a for the bigram model: 0.010000000000000002


In [ ]:
a = []
cr_entr = []

for i in np.arange(0.001, 1.001, 0.001):
    hc, pp = tri_model.entr_perp(dev_set, a=i)
    a.append(i)
    cr_entr.append(hc)

print(f'Best a for the trigram model: {a[np.argmin(cr_entr)]}')

Best a for the trigram model: 0.007


In [ ]:
lap_hc, lap_pp = bi_model.entr_perp(test_set, a=0.01)
kn_hc, kn_pp = bi_model.entr_perp(test_set, smoothing='kn')
tri_hc, tri_pp = tri_model.entr_perp(test_set, a=0.007)

print(f"Cross-Entropy and Perplexity using the Bigram Model with Add-a Smoothing: {lap_hc} and {lap_pp}")
print(f"Cross-Entropy and Perplexity using the Bigram Model with K-N Smoothing: {kn_hc} and {kn_pp}")
print(f"Cross-Entropy and Perplexity using the Trigram Model with Add-a Smoothing: {tri_hc} and {tri_pp}")

Cross-Entropy and Perplexity using the Bigram Model with Add-a Smoothing: 6.8930890717714455 and 118.8574951123271
Cross-Entropy and Perplexity using the Bigram Model with K-N Smoothing: 6.529238290883772 and 92.3626903053094
Cross-Entropy and Perplexity using the Trigram Model with Add-a Smoothing: 8.728574636695134 and 424.1923040644523


In [ ]:
from random import shuffle
test_shfl = test_set[:]
foo = [shuffle(sent) for sent in test_shfl]  # shuffle the order of words from each sentence

In [ ]:
lap_hc, lap_pp = bi_model.entr_perp(test_shfl, a=0.01)
kn_hc, kn_pp = bi_model.entr_perp(test_shfl, smoothing='kn')
tri_hc, tri_pp = tri_model.entr_perp(test_shfl, a=0.007)

print(f"Cross-Entropy and Perplexity using the Bigram Model with Add-a Smoothing on shuffled sentences: {lap_hc} and {lap_pp}")
print(f"Cross-Entropy and Perplexity using the Bigram Model with K-N Smoothing on shuffled sentences: {kn_hc} and {kn_pp}")
print(f"Cross-Entropy and Perplexity using the Trigram Model with Add-a Smoothing on shuffled sentences: {tri_hc} and {tri_pp}")

Cross-Entropy and Perplexity using the Bigram Model with Add-a Smoothing on shuffled sentences: 9.550154295720343 and 749.6920513343857
Cross-Entropy and Perplexity using the Bigram Model with K-N Smoothing on shuffled sentences: 8.386838674787834 and 334.72643331454844
Cross-Entropy and Perplexity using the Trigram Model with Add-a Smoothing on shuffled sentences: 10.894391388165639 and 1903.4373735979686


In [ ]:
import pandas as pd

def following_bi(word, model, smoother='kn', a=1):  # This function will return the top 10 most probable word continuations
    voc = list(model.unigram_voc_.keys())
    voc.remove('*start*')
    voc.remove('*UNK*')
    voc.remove('*end*')
    
    if smoother == 'kn':
        next_word = {key: bi_model.kn_prob((word, key)) for key in voc}
    else:
        next_word = {key: bi_model.estimate_ngram_prob((word, key), a=a) for key in voc}

    sorted_words = dict(sorted(next_word.items(), key=lambda item: item[1]))
    top10 = {i: sorted_words[i] for i in list(sorted_words.keys())[-10:]}

    return pd.DataFrame(top10.items()).rename(columns = {0: word, 1: 'Probability'}).sort_values(by='Probability', ascending=False)
    

In [ ]:
following_bi('he', bi_model)

,he,Probability
9,said,0.393450
8,says,0.334239
7,is,0.042016
6,has,0.019870
5,was,0.014306
4,will,0.012981
3,s,0.012233
2,had,0.008396
1,and,0.007996
0,also,0.007358


In [ ]:
following_bi('good', bi_model)

,good,Probability
9,news,0.097393
8,for,0.032351
7,at,0.027960
6,to,0.027091
5,and,0.024719
4,as,0.017084
3,prices,0.015524
2,enough,0.015372
1,thing,0.015266
0,on,0.013823


In [ ]:
following_bi('make', bi_model)

,make,Probability
9,a,0.126875
8,the,0.121068
7,it,0.114708
6,sure,0.077510
5,up,0.046265
4,them,0.030279
3,sense,0.016443
2,any,0.014325
1,an,0.012470
0,people,0.012261


### Beam Search

In [ ]:
import difflib
import math

In [ ]:
len(b_model.unigram_voc_)

5542

In [ ]:
#this method calculates distances between words, sort them and returns the N closest ones
def close_words(sentence): 
  choose = len(bi_model.unigram_voc_)#hyper parameter
  close = [['*start*']*choose]
  
  voc = list(bi_model.unigram_voc_.keys())
  voc.remove('*UNK*')
  voc.remove('*start*')
  voc.remove('*end*')

  for s in sentence[1:-1]:  
    dis = []
    for i in voc:

      #dis.append((i,1/(editdistance.eval(i, s)+1))) #Levensein distance
      dis.append((i,difflib.SequenceMatcher(None, i, s).ratio())) #gestalt pattern matching
    dis.sort(key=lambda x: x[1],reverse = True) #sort based on distance
    close.append(dis[:choose])
  close.append(['*end*']*choose)

  return close

#this method set a score a sequence according to the formula given
def prob_calc(sent,true):
  sent = sent.split()
  l1 = 0.9 #hyper parameter
  l2 = 1 - l1 #hyper parameter

  S = 0
  for i in range(len(sent)) :
    #S -= math.log(1/(editdistance.eval(sent[i], true[i])+1),2)  
    S -= math.log(difflib.SequenceMatcher(None, sent[i], true[i]).ratio()+0.0001,2)

  return l1*S-l2*bi_model.estimate_sent_prob([sent], padding=False)[0] #calculate bigram and spelling probability

#this is the beam search algorithm
def beam_search(list_sen,close):
  if(len(list_sen) == 2):   #if there are only 2 words left in the sequence
    calc_prob = []  
    print(list_sen)
    for j in close[len(list_sen)-1]: #find the closest words              
      calc_prob.append((list_sen[0],j[0],prob_calc(list_sen[0]+' '+j[0],list_sen)))
    print('fin')
    calc_prob.sort(key=lambda x: x[2])   #sort according to score
    #print(calc_prob) 

  else :  #if there are mroe than two words in the sequence
    calc_prob = []  
    print(list_sen)
    prev = beam_search(list_sen[0:-1],close) #call itslef until there are only two words
    for p in prev :  #from the result of the previous score    
      for j in close[len(list_sen)-1]:  #for every new word            
        calc_prob.append((p,j[0],prob_calc(p+' '+j[0],list_sen)))#calculate probability of the sequence plus the extra word
    calc_prob.sort(key=lambda x: x[2])#sort based on socre
  print( [calc_prob[0][0]+' '+calc_prob[0][1],calc_prob[1][0]+' '+calc_prob[1][1]])
  return  [calc_prob[0][0]+' '+calc_prob[0][1],calc_prob[1][0]+' '+calc_prob[1][1]]#return top two sequnces

The football test case taken from lecture

In [ ]:
test = "*start* he plas god ftbal *end*".split()
beam_search(test,close_words(test))

['*start*', 'he', 'plas', 'god', 'ftbal', '*end*']
['*start*', 'he', 'plas', 'god', 'ftbal']
['*start*', 'he', 'plas', 'god']
['*start*', 'he', 'plas']
['*start*', 'he']
fin
['*start* he', '*start* the']
['*start* the past', '*start* the last']
['*start* the past good', '*start* the past gold']
['*start* the past gold fatal', '*start* the past good fatal']
['*start* the past gold fatal *', '*start* the past gold fatal *']


['*start* the past gold fatal *', '*start* the past gold fatal *']

Create random test cases taken from the test set

In [ ]:
import random 

false_sents = []  # Randomly remove a letter from each token
for sent in test_set:
    clean = LM.word_cleaner(sent)
    false = []
    for token in clean:
        if len(token) > 1:
            remove_ind = random.randint(0, len(token)-1)
            false.append(token[:remove_ind] + token[remove_ind+1:])
        else:
            false.append(token)
    false_sents.append(false)


This is a test case encountered during testing 

In [ ]:
test = ['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't', 'may', 'ther', 'mammal', 'tt', 's', 'vry', 'imple', 'end*']

In [ ]:
beam_search(test,close_words(test))

['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't', 'may', 'ther', 'mammal', 'tt', 's', 'vry', 'imple', 'end*']
['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't', 'may', 'ther', 'mammal', 'tt', 's', 'vry', 'imple']
['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't', 'may', 'ther', 'mammal', 'tt', 's', 'vry']
['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't', 'may', 'ther', 'mammal', 'tt', 's']
['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't', 'may', 'ther', 'mammal', 'tt']
['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't', 'may', 'ther', 'mammal']
['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't', 'may', 'ther']
['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't', 'may']
['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't']
['*start*', 'w', 'cn', 'appl', 'his', 'metod']
['*start*', 'w', 'cn', 'appl', 'his']
['*start*', 'w', 'cn', 'appl']
['*start*', 'w', 'cn']
['*start*', 'w']
fin
['*start* we', '*start* wa']
['*start* we can', '*start* we cent']
['*start* we can apply', '*start* we

['*start* we can apply this method to many other mammals that is very simple *',
 '*start* we can apply this method to many other mammals that is very simple *']

This is the same case after changing one word. Notice the difference in the plurality of mammals due to the change.

In [ ]:
test = ['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't', 'may', 'a', 'mammal', 'tt', 's', 'vry', 'imple', 'end*']
beam_search(test,close_words(test))

['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't', 'may', 'a', 'mammal', 'tt', 's', 'vry', 'imple', 'end*']
['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't', 'may', 'a', 'mammal', 'tt', 's', 'vry', 'imple']
['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't', 'may', 'a', 'mammal', 'tt', 's', 'vry']
['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't', 'may', 'a', 'mammal', 'tt', 's']
['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't', 'may', 'a', 'mammal', 'tt']
['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't', 'may', 'a', 'mammal']
['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't', 'may', 'a']
['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't', 'may']
['*start*', 'w', 'cn', 'appl', 'his', 'metod', 't']
['*start*', 'w', 'cn', 'appl', 'his', 'metod']
['*start*', 'w', 'cn', 'appl', 'his']
['*start*', 'w', 'cn', 'appl']
['*start*', 'w', 'cn']
['*start*', 'w']
fin
['*start* we', '*start* wa']
['*start* we can', '*start* we cent']
['*start* we can apply', '*start* we can apple']
['*start

['*start* we can apply this method to pay a mammal that is very simple *',
 '*start* we can apply this method to pay a mammal that is very simple *']

This picks a random test case

In [ ]:
ind = random.randint(0, len(false_sents))
test = ['*start*', *false_sents[ind], '*end*']
beam_search(test,close_words(test))
print(f'Correct Sentence: {LM.word_cleaner(test_set[ind])}')

['*start*', 'toay', 'th', 'mazon', 'lows', 'ino', 'te', 'atlatic', 'ocan', '*end*']
['*start*', 'toay', 'th', 'mazon', 'lows', 'ino', 'te', 'atlatic', 'ocan']
['*start*', 'toay', 'th', 'mazon', 'lows', 'ino', 'te', 'atlatic']
['*start*', 'toay', 'th', 'mazon', 'lows', 'ino', 'te']
['*start*', 'toay', 'th', 'mazon', 'lows', 'ino']
['*start*', 'toay', 'th', 'mazon', 'lows']
['*start*', 'toay', 'th', 'mazon']
['*start*', 'toay', 'th']
['*start*', 'toay']
fin
['*start* today', '*start* to']
['*start* to the', '*start* today the']
['*start* to the main', '*start* to the moon']
['*start* to the main flows', '*start* to the moon flows']
['*start* to the main flows in', '*start* to the moon flows in']
['*start* to the main flows in the', '*start* to the moon flows in the']
['*start* to the main flows in the atlantic', '*start* to the moon flows in the atlantic']
['*start* to the main flows in the atlantic ocean', '*start* to the moon flows in the atlantic ocean']
['*start* to the main flows in